## `Influence of Machine Learning on the Diagnosis of Cardiovascular Diseases`

### `Problem Statement`

Cardiovascular Diseases (CVDs) are considered the leading cause of mortality globally, with at least `80%` of deaths associated with heart disease and strokes in individuals below the age of `70 years` (Rana et al., 2025). Indications that a patient has CVD are high levels of biomarkers in their blood, significant chest pain and abnormal readings on an ECG. Surprisingly, a lot of patients with CVD are difficult to diagnose using ECG when their results are largely normal. Therefore, the use of AI approaches can make a significant difference in how we diagnose CVD, ultimately improving the outcomes.

### `Objectives`

#### `General Objective`
- To develop a machine learning predictive model to aid in the diagnosis of cardiovascular diseases.

#### `Specific Objectives`

- To examine the appliations of machine learning in the diagnosis of CVDs.
- To evaluate the accuracy of machine learning tools in the diagnosis of CVDs.
- To design a machine learning model to diagnose CVDs.
- To evaluate and validate the developed CVDs diagnosing machine learning model using performance metrics.

### `Data Understanding`